## PyTorch Resnets

In [42]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..", "..")))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH,
)
from experiments.utils.parser import Parser

In [52]:
series = 81
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = "resnet-human"
series_path = os.path.join(NODE_PROFILING_RESULTS_PATH, "series", str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper, model_name=model_name
)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [44]:
# results = loader._read_results(selected)
# final_dataframe = []
# for experiment_id, result in results.items():
#     processed_exp = {'experiment_id': int(experiment_id)}
#     flattened_results = self.flatten_results(
#         results[str(experiment_id)]['responses'])
#     if log is not None:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log[experiment_id])
#     else:
#         latencies, timeout_count = self.latency_calculator(
#             flattened_results, log)

In [45]:
results.columns

Index(['experiment_id', 'client_to_model_avg',
       'client_to_model_p99', 'client_to_model_p50',
       'client_to_model_var', 'client_to_model_max',
       'client_to_model_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_client_avg', 'model_to_client_p99',
       'model_to_client_p50', 'model_to_client_var',
       'model_to_client_max', 'model_to_client_min',
       'start_time', 'end_time', 'duration', 'timeout_count',
       'cpu_usage_count_avg', 'cpu_usage_count_p99', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p99', 'cpu_usage_rate_p50',
       'cpu_usage_rate_var', 'cpu_usage_rate_max', 'cpu_usage_rate_min',
       'cpu_throttled_count_avg', 'cpu_throttled_count_p99',
       'cpu_throttled_count_p50', 'cpu_throttled_count_var',
       'cpu_throttled_co

In [46]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config)

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1'],
    'data_type': 'image',
    'max_batch_size': ['1', '16'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['resnet18'],
    'node_name': 'resnet-human',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1, 2],
    'series': 81,
    'series_meta': 'test for the queue 1,\\n cpu type: Intel(R) Xeon(R) Gold '
                   '6126 CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 300, 'loads_to_test': [100]},
    'workload_type': 'static'}


In [47]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,resnet-human,resnet18,1,10Gi,1,1,100,300,81,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
1,2,video,resnet-human,resnet18,1,10Gi,1,1,100,300,81,"test for the queue 1,\n cpu type: Intel(R) Xeo...",2,False,exponential,image,1
2,3,video,resnet-human,resnet18,1,10Gi,16,1,100,300,81,"test for the queue 1,\n cpu type: Intel(R) Xeo...",1,False,exponential,image,1
3,4,video,resnet-human,resnet18,1,10Gi,16,1,100,300,81,"test for the queue 1,\n cpu type: Intel(R) Xeo...",2,False,exponential,image,1


In [1]:
experiment_ids = key_config_df["experiment_id"].tolist()
metadata_columns = [
    "model_variant",
    "max_batch_size",
    "cpu_request",
    "load",
    "load_duration",
    "replicas",
]
results_columns = [
    "timeout_count",
    "model_latencies_min",
    "model_latencies_p99",
    "cpu_usage_count_avg",
    "model_latencies_avg",
    "throughput_max",
]
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns,
)
output["total_requests"] = output["load"] * output["load_duration"]
# output.sort_values(by='throughput')
# display(output[['model_variant', 'max_batch_size', 'cpu_request', 'load', 'load_duration', 'replicas', 'total_requests', 'timeout_count', 'model_latencies_min', 'model_latencies_p99', 'model_latencies_avg', 'throughput_max']])
display(output[output["load"] == 80])

NameError: name 'key_config_df' is not defined

In [58]:
key_config_df.columns

Index(['experiment_id', 'pipeline_name', 'node_name', 'model_variant',
       'cpu_request', 'memory_request', 'max_batch_size', 'max_batch_time',
       'load', 'load_duration', 'series', 'series_meta', 'replicas',
       'no_engine', 'mode', 'data_type', 'benchmark_duration'],
      dtype='object')